# Manual Checks
* Cleaning the "Project" and "Allocation" sheet have manual components. 
    * `clean_allocation_manual()` and `clean_project_manual()` respectively. 
* This notebook walks through each manual step and any changes.

In [1]:
import A1_data_prep
import A5_crosswalks as crosswalks
import A6_zev
# import numpy as np
import pandas as pd
from calitp_data_analysis.sql import to_snakecase


pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Project Sheet

In [2]:
# Loading in the original sheet that has not been manipulated
project = A1_data_prep.load_project()

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
# Check award cycles to ensure they are in the format of 1,2,3,4,5.
# If not, replace this in A1_data_prep's def clean_project_manual() function.
project.award_cycle.value_counts()

3           28
5           23
4           16
1           14
2           14
FY 21/22     1
Name: award_cycle, dtype: int64

In [4]:
# Check award years to ensure the projects line up with expectations
project.award_year.value_counts()

2018    28
2022    23
2020    17
2015    14
2016    14
Name: award_year, dtype: int64

### Grant Recipients
* Check grant recipients and ensure they only appear once.
* Add duplicates in the A5_crosswalks.py file.

In [5]:
# First pass: clean up org names with a function first.
# This function is applied in A1_data_prep's clean_project() function.
project = A1_data_prep.organization_cleaning(project, 'grant_recipient')

In [6]:
# Check grant recipients and ensure they only appear under one name.
# If there are duplicates, replace them in A5_crosswalks.
project.grant_recipient.sort_values().unique().tolist()

['Alameda Contra Costa Transit District',
 'Anaheim Transportation Network',
 'Antelope Valley Transit Authority',
 'Bay Area Rapid Transit',
 'Capitol Corridor Joint Powers Authority',
 'City Of Cupertino',
 'City Of Fresno',
 'City Of Glendale',
 'City Of Inglewood',
 'City Of Los Angeles',
 'City Of Oakland',
 'City Of Santa Monica',
 'City Of Torrance',
 'City Of Wasco',
 'Contra Costa Transportation Authority',
 'Foothill Transit',
 'Fresno County Rural Transit Agency',
 'Humboldt Transit Authority',
 'Lake Transit Authority',
 'Livermore Amador Valley Transit Authority',
 'Long Beach Transit',
 'Los Angeles County Metropolitan Transportation Authority',
 'Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency',
 'Monterey-Salinas Transit District',
 'Orange County Transportation Authority',
 'Peninsula Corridor Joint Powers Board',
 'Sacramento Regional Transit District',
 'San Bernardino County Transportation Authority',
 'San Diego Association Of Governments',
 'San Diego M

### Check PPNO - Project Relationship

In [7]:
f"There are {project.ppno.nunique()} PPNO numbers and {project.project_title.nunique()} project titles."

'There are 90 PPNO numbers and 95 project titles.'

In [8]:
# Check PPNO with more than one project.
# Make sure they make sense. 
ppno_df = (
    A6_zev.value_counts_df(project, "ppno")
    .rename(columns={"ppno": "total projects", "index": "ppno"})
)

In [9]:
ppno_df = ppno_df.loc[ppno_df['total projects'] > 1]

In [10]:
ppno_more_than_1_project = ppno_df.ppno.tolist()

In [11]:
project[["award_year","ppno", "project_title", "grant_recipient", ]][
    project["ppno"].isin(ppno_more_than_1_project)
].sort_values('ppno')

,award_year,ppno,project_title,grant_recipient
9,2015,CP006,SFMTA Light Rail Vehicle Fleet Expansion,San Francisco Municipal Transportation Agency
25,2016,CP006,Light Rail Modernization and Expansion Program,San Francisco Municipal Transportation Agency
45,2018,CP006,Transit Capacity Expansion Program,San Francisco Municipal Transportation Agency
38,2018,CP031,All Aboard 2018: Transforming SoCal Rail Travel,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency
39,2018,CP031,Building Up: LOSSAN North Improvement Program,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency
42,2018,CP034,Diesel Multiple Unit Vehicle to Zero- or Low-Emission Vehicle Conversion and West Valley Connector Bus Rapid Transit,San Bernardino County Transportation Authority
44,2018,CP034,Blue Line Rail Corridor Transit Enhancements,San Diego Metropolitan Transit System
27,2016,CP057,BART Silicon Valley Phase II Extension,Santa Clara Valley Transportation Authority
50,2018,CP057,"VTA’s BART Silicon Valley Extension, Phase II",Santa Clara Valley Transportation Authority


## Allocation Sheet

In [12]:
# Load in untouched allocation sheet
allocation = A1_data_prep.load_allocation()

In [13]:
allocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529 entries, 0 to 528
Data columns (total 52 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   award_year                              522 non-null    float64       
 1   project_#                               515 non-null    float64       
 2   grant_recipient                         515 non-null    object        
 3   implementing_agency                     515 non-null    object        
 4   ppno                                    511 non-null    object        
 5   project_id                              293 non-null    float64       
 6   ea                                      289 non-null    object        
 7   components                              515 non-null    object        
 8   phase                                   515 non-null    object        
 9   allocation_amount                       529 non-null  

In [14]:
# Clean up grant recipient names
allocation = A1_data_prep.organization_cleaning(allocation, 'grant_recipient')

### Dates
* Correct any values that didn't read in correctly in `A5_crosswalks`.
* String values without date information such as "pending" or "on hold" don't need to be corrected as they are coerced later on. 
* In contrast values such as "6/30/2021\n12/31/2021\n10/20/2022" should be corrected. 

In [15]:
allocation._3rd_party_award_date.unique().tolist()

[datetime.datetime(2016, 3, 14, 0, 0),
 datetime.datetime(2016, 6, 1, 0, 0),
 datetime.datetime(2017, 6, 28, 0, 0),
 datetime.datetime(2016, 11, 3, 0, 0),
 datetime.datetime(2015, 11, 30, 0, 0),
 datetime.datetime(2015, 10, 21, 0, 0),
 datetime.datetime(2016, 1, 8, 0, 0),
 '-',
 datetime.datetime(2017, 5, 30, 0, 0),
 datetime.datetime(2016, 11, 1, 0, 0),
 datetime.datetime(2015, 1, 1, 0, 0),
 datetime.datetime(2018, 2, 2, 0, 0),
 datetime.datetime(2017, 6, 23, 0, 0),
 datetime.datetime(2015, 10, 15, 0, 0),
 datetime.datetime(2016, 4, 6, 0, 0),
 datetime.datetime(2017, 3, 22, 0, 0),
 datetime.datetime(2017, 5, 16, 0, 0),
 datetime.datetime(2021, 3, 18, 0, 0),
 datetime.datetime(2019, 4, 19, 0, 0),
 nan,
 datetime.datetime(2022, 7, 26, 0, 0),
 datetime.datetime(2019, 2, 13, 0, 0),
 datetime.datetime(2019, 1, 9, 0, 0),
 datetime.datetime(2021, 8, 9, 0, 0),
 datetime.datetime(2022, 2, 15, 0, 0),
 datetime.datetime(2017, 1, 13, 0, 0),
 datetime.datetime(2021, 9, 22, 0, 0),
 datetime.datetim

In [16]:
allocation.phase_completion_date.unique().tolist()

[datetime.datetime(2022, 3, 30, 0, 0),
 'Complete\n6/1/2019',
 datetime.datetime(2021, 6, 30, 0, 0),
 'Complete\n6/11/2018',
 'Complete\n2/11/2018',
 'Complete\n6/30/2020',
 datetime.datetime(2020, 9, 30, 0, 0),
 datetime.datetime(2018, 6, 30, 0, 0),
 datetime.datetime(2020, 6, 29, 0, 0),
 datetime.datetime(2019, 11, 1, 0, 0),
 datetime.datetime(2018, 12, 10, 0, 0),
 'Complete\n11/13/2019',
 datetime.datetime(2020, 3, 30, 0, 0),
 'Sep-22',
 datetime.datetime(2021, 12, 30, 0, 0),
 datetime.datetime(2021, 9, 30, 0, 0),
 datetime.datetime(2020, 5, 16, 0, 0),
 '-',
 datetime.datetime(2020, 6, 30, 0, 0),
 '6/30/2020\nComplete',
 'Jun-24',
 'Dec-25',
 '6/30/2022\nComplete',
 'Sep-24',
 'Jul-24',
 datetime.datetime(2018, 5, 26, 0, 0),
 datetime.datetime(2019, 5, 21, 0, 0),
 datetime.datetime(2022, 1, 16, 0, 0),
 'Oct-22',
 datetime.datetime(2021, 12, 31, 0, 0),
 datetime.datetime(2018, 2, 1, 0, 0),
 'Jan-24',
 'Apr-24',
 datetime.datetime(2021, 7, 7, 0, 0),
 datetime.datetime(2020, 5, 7, 0, 0

In [17]:
allocation.allocation_date.unique().tolist()

[datetime.datetime(2015, 10, 22, 0, 0),
 datetime.datetime(2016, 5, 19, 0, 0),
 datetime.datetime(2016, 6, 30, 0, 0),
 datetime.datetime(2015, 12, 10, 0, 0),
 datetime.datetime(2015, 8, 27, 0, 0),
 datetime.datetime(2016, 1, 21, 0, 0),
 datetime.datetime(2017, 6, 29, 0, 0),
 datetime.datetime(2016, 10, 20, 0, 0),
 datetime.datetime(2017, 8, 17, 0, 0),
 datetime.datetime(2018, 2, 1, 0, 0),
 datetime.datetime(2016, 3, 16, 0, 0),
 datetime.datetime(2016, 3, 17, 0, 0),
 datetime.datetime(2017, 1, 19, 0, 0),
 datetime.datetime(2020, 10, 22, 0, 0),
 datetime.datetime(2018, 10, 18, 0, 0),
 datetime.datetime(2022, 1, 27, 0, 0),
 datetime.datetime(2018, 8, 16, 0, 0),
 datetime.datetime(2017, 3, 16, 0, 0),
 datetime.datetime(2021, 6, 24, 0, 0),
 datetime.datetime(2020, 6, 25, 0, 0),
 datetime.datetime(2016, 12, 8, 0, 0),
 datetime.datetime(2020, 6, 24, 0, 0),
 datetime.datetime(2019, 12, 5, 0, 0),
 nan,
 datetime.datetime(2021, 1, 28, 0, 0),
 datetime.datetime(2018, 5, 17, 0, 0),
 datetime.datet

In [18]:
allocation.led.unique().tolist()

[datetime.datetime(2022, 3, 31, 0, 0),
 datetime.datetime(2019, 6, 1, 0, 0),
 datetime.datetime(2020, 6, 28, 0, 0),
 datetime.datetime(2021, 6, 30, 0, 0),
 datetime.datetime(2019, 11, 3, 0, 0),
 datetime.datetime(2018, 11, 30, 0, 0),
 datetime.datetime(2020, 6, 30, 0, 0),
 datetime.datetime(2019, 1, 8, 0, 0),
 datetime.datetime(2018, 6, 30, 0, 0),
 datetime.datetime(2020, 6, 29, 0, 0),
 datetime.datetime(2019, 11, 1, 0, 0),
 datetime.datetime(2018, 12, 10, 0, 0),
 datetime.datetime(2021, 2, 2, 0, 0),
 datetime.datetime(2020, 6, 23, 0, 0),
 datetime.datetime(2022, 9, 30, 0, 0),
 'October 15, 2018\nSeptember 30, 2021',
 datetime.datetime(2021, 9, 22, 0, 0),
 datetime.datetime(2020, 5, 16, 0, 0),
 datetime.datetime(2023, 10, 21, 0, 0),
 datetime.datetime(2023, 6, 30, 0, 0),
 datetime.datetime(2025, 1, 27, 0, 0),
 datetime.datetime(2024, 6, 9, 0, 0),
 datetime.datetime(2024, 6, 24, 0, 0),
 datetime.datetime(2025, 2, 17, 0, 0),
 datetime.datetime(2022, 12, 30, 0, 0),
 datetime.datetime(2024

## Check PPNO between Allocation and Project Sheet
* Use the PPNO off of "Project Sheet" as the source of truth.

In [19]:
# PPNO list for project
project_ppno_list = project.ppno.unique().tolist()

In [20]:
# PPNO list for allocation
allocation_ppno_list = allocation.ppno.unique().tolist()

In [21]:
# Check lengths 
len(project_ppno_list) == len(allocation_ppno_list)

False

In [22]:
# Check differences between allocation vs ppno list
ppno_unique_to_alloc = list(set(allocation_ppno_list) - set(project_ppno_list))

In [23]:
len(ppno_unique_to_alloc)

9

In [24]:
# Check out the projects that appear in alloc sheet but not projects. 
projects_found_only_in_alloc = (allocation[["award_year", "grant_recipient", "ppno"]][
    allocation["ppno"].isin(ppno_unique_to_alloc)
]).drop_duplicates().reset_index(drop = True)

In [25]:
projects_found_only_in_alloc

,award_year,grant_recipient,ppno
0,2015.0,Southern California Regional Rail Authority,CP002
1,2016.0,Antelope Valley Transit Authority,CP018
2,2016.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP024
3,2016.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP021
4,2018.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP301
5,2018.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP042
6,2018.0,Sacramento Regional Transit District,CP053
7,2018.0,San Diego Metropolitan Transit System,CP032
8,2018.0,Transportation Agency For Monterey County,1155N


In [26]:
# Merge df above with a subset of the project df to figure out what's going on. 
project_subset = ['award_year','grant_recipient','project_title','ppno']
m1 = pd.merge(projects_found_only_in_alloc, project[project_subset], how="left", on = ['grant_recipient','award_year'], suffixes = ('_alloc','_project'), indicator = True)

In [27]:
# For projects that natched over, correct ppno_alloc values to ppno_project
m1[m1['project_title'].notnull()].drop_duplicates()

,award_year,grant_recipient,ppno_alloc,project_title,ppno_project,_merge
1,2016.0,Antelope Valley Transit Authority,CP018,"Zero Emission Bus and Vanpool Expansion in the Antelope Valley, Kern County and the Coachella Valley",CP019,both
2,2016.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP024,All Aboard: Transforming Southern California Rail Travel,CP043,both
3,2016.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP021,All Aboard: Transforming Southern California Rail Travel,CP043,both
4,2018.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP301,All Aboard 2018: Transforming SoCal Rail Travel,CP031,both
5,2018.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP301,Building Up: LOSSAN North Improvement Program,CP031,both
6,2018.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP042,All Aboard 2018: Transforming SoCal Rail Travel,CP031,both
7,2018.0,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,CP042,Building Up: LOSSAN North Improvement Program,CP031,both
8,2018.0,Sacramento Regional Transit District,CP053,Accelerating Rail Modernization and Expansion in the Capital Region,CP052,both
9,2018.0,San Diego Metropolitan Transit System,CP032,Blue Line Rail Corridor Transit Enhancements,CP034,both
10,2018.0,Transportation Agency For Monterey County,1155N,Extend rail service to Monterey County,1155A,both


In [28]:
# Find the ppno for the rows missing values from the Project sheet after merging
missing_m1 = m1[m1['project_title'].isnull()]
missing_m1

,award_year,grant_recipient,ppno_alloc,project_title,ppno_project,_merge
0,2015.0,Southern California Regional Rail Authority,CP002,NaN,NaN,left_only


In [29]:
# Get the year of missing projects
missing_m1_years = (missing_m1.award_year.unique().tolist()) 

In [30]:
# Filter based on year
project[project_subset][
    project["award_year"].isin(missing_m1_years)
]

,award_year,grant_recipient,project_title,ppno
0,2015,Antelope Valley Transit Authority,Regional Transit Interconnectivity & Environmental Sustability,CP005
1,2015,Capitol Corridor Joint Powers Authority,Travel Time Reduction Project,CP012
2,2015,Los Angeles County Metropolitan Transportation Authority,Willowbrook/Rosa Parks Station & Blue Line Light Rail Operational Improvements Project,CP015
3,2015,Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency,Pacific Surfliner Transit Transfer Program,CP007
4,2015,Monterey-Salinas Transit District,Monterey Bay Operations and Maintenance Facility/Salinas Transit Service Project,CP013
5,2015,Orange County Transportation Authority,Bravo! Route 560 Rapid Buses,CP004
6,2015,Sacramento Regional Transit District,Sacramento Regional Transit's Refurbishment of 7 Light Rail Vehicles,CP001
7,2015,San Diego Association Of Governments,South Bay Bus Rapid Transit,CP003
8,2015,San Diego Metropolitan Transit System,San Diego Metropolitan Transit System Trolley Capacity Improvements,CP008
9,2015,San Francisco Municipal Transportation Agency,SFMTA Light Rail Vehicle Fleet Expansion,CP006
